In [8]:
import json
import numpy as np
from scipy.optimize import minimize
from itertools import combinations

def load_materials_from_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def get_user_constraints():
    return {
        'target_density': float(input("Enter target density: ")),
        'target_strength': float(input("Enter minimum tensile strength: ")),
        'target_corrosion_resistance': float(input("Enter minimum corrosion resistance: ")),
        'target_cost': float(input("Enter maximum allowable cost: ")),
        'target_sustainability': float(input("Enter target sustainability score: "))
    }

def normalize(data):
    data = np.array(data)
    data_min, data_max = np.min(data), np.max(data)
    return np.ones_like(data) if data_max == data_min else (data - data_min) / (data_max - data_min)

def objective(x, materials, normalized_properties, constraints, weights):
    composition_penalty = abs(sum(x) - 1) * 1000
    
    properties = ['density', 'strength', 'corrosion_resistance', 'cost', 'sustainability_score']
    property_penalties = sum(
        weights[prop] * (sum(x[i] * normalized_properties[prop][i] for i in range(len(materials))) - normalize([constraints[f'target_{prop}']])[0])**2
        for prop in properties
    )
    
    return composition_penalty + property_penalties

def optimize_composition(materials, constraints, weights):
    property_names = ['density', 'strength', 'corrosion_resistance', 'cost', 'sustainability_score']
    properties = {prop: [materials[mat][prop] for mat in materials] for prop in property_names}
    normalized_properties = {prop: normalize(properties[prop]) for prop in property_names}
    
    bounds = [(0, 1) for _ in range(len(materials))]
    constraints_eq = {'type': 'eq', 'fun': lambda x: sum(x) - 1}
    
    result = minimize(
        objective, 
        [1/len(materials)] * len(materials), 
        args=(materials, normalized_properties, constraints, weights),
        method='SLSQP', 
        bounds=bounds, 
        constraints=[constraints_eq]
    )
    
    return result.x if result.success else None

def main():
    file_path = r"C:\Users\olitu\OneDrive\Documents\Python\Dataset TATA INNOVENT cc.json"
    all_materials = load_materials_from_json(file_path)
    constraints = get_user_constraints()
    
    weights = {
        'density': 0.5,
        'strength': 1.5,
        'corrosion_resistance': 1.0,
        'cost': 2.0,
        'sustainability_score': 2.5
    }
    
    # Filter materials based on constraints
    filtered_materials = {
        name: props for name, props in all_materials.items()
        if props['cost'] <= constraints['target_cost'] and props['sustainability_score'] >= constraints['target_sustainability']
    }
    
    best_composition = None
    best_score = float('inf')
    best_materials = None
    
    # Try all combinations of 2 and 3 materials
    for r in [2, 3]:
        for materials_subset in combinations(filtered_materials, r):
            materials = {name: filtered_materials[name] for name in materials_subset}
            composition = optimize_composition(materials, constraints, weights)
            
            if composition is not None:
                score = objective(composition, materials, 
                                  {prop: normalize([materials[mat][prop] for mat in materials]) for prop in weights.keys()},
                                  constraints, weights)
                
                if score < best_score:
                    best_score = score
                    best_composition = composition
                    best_materials = materials
    
    if best_composition is not None:
        print("Optimal Alloy Composition (in percentages):")
        for mat, percentage in zip(best_materials.keys(), best_composition):
            print(f"{mat}: {percentage * 100:.2f}%")
    else:
        print("Optimization failed to find a solution.")

if __name__ == "__main__":
    main()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 637: character maps to <undefined>